In [1]:
# make sure ES is up and running
import requests
res = requests.get('http://localhost:9200')
print(res.content)

{
  "name" : "38KW7e4",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "X7_Qa3ghQ5yPHNUzSox3ug",
  "version" : {
    "number" : "5.4.0",
    "build_hash" : "780f8c4",
    "build_date" : "2017-04-28T17:43:27.229Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.0"
  },
  "tagline" : "You Know, for Search"
}



In [2]:
#connect to our cluster
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [3]:
# Database
from humbledb import Mongo, Document
class MovieDoc(Document):
    config_database = 'emdb'
    config_collection = 'movies'

In [7]:
with Mongo:
    for m in MovieDoc.find():
        content = m.for_json()
        content.pop('_id', None)
        es.index(index='emdb', doc_type='movies', id=m['_id'], body=content)

In [3]:
es.get(index='emdb', doc_type='movies', id='592571528fbb650bf6cea4da')

{u'_id': u'592571528fbb650bf6cea4da',
 u'_index': u'emdb',
 u'_source': {u'Cast': u'Christian Bale (Bruce Wayne), Heath Ledger (Joker), Aaron Eckhart (Harvey Dent), Michael Caine (Alfred), Maggie Gyllenhaal (Rachel)',
  u'Country': u'USA, UK',
  u'Director': u'Christopher Nolan',
  u'Genres': u'Action, Crime, Drama, Thriller',
  u'Language': u'English, Mandarin',
  u'Plot': u'Set within a year after the events of Batman Begins, Batman, Lieutenant James Gordon, and new district attorney Harvey Dent successfully begin to round up the criminals that plague Gotham City until a mysterious and sadistic criminal mastermind known only as the Joker appears in Gotham, creating a new wave of chaos. Batman\'s struggle against the Joker becomes deeply personal, forcing him to "confront everything he believes" and improve his technology to stop him. A love triangle develops between Bruce Wayne, Dent and Rachel Dawes.',
  u'Rating': 9.0,
  u'Runtime': u'152',
  u'Title': u'Dark Knight, The (2008)'},


In [21]:
query = {"query" : { "match_phrase" : { "Title" :  "Dark Knight Rises" }}, "highlight" : { "fields" : { "Title" :  {}}} }
es.search(index="emdb", body=query)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'592572248fbb650bf6cea515',
    u'_index': u'emdb',
    u'_score': 8.52379,
    u'_source': {u'Cast': u'Christian Bale (Bruce Wayne), Gary Oldman (Commissioner Gordon), Tom Hardy (Bane), Joseph Gordon-Levitt (Blake), Anne Hathaway (Selina)',
     u'Country': u'UK, USA',
     u'Director': u'Christopher Nolan',
     u'Genres': u'Action, Thriller',
     u'Language': u'English, Arabic',
     u'Plot': u"Despite his tarnished reputation after the events of The Dark Knight, in which he took the rap for Dent's crimes, Batman feels compelled to intervene to assist the city and its police force which is struggling to cope with Bane's plans to destroy the city.",
     u'Rating': 8.5,
     u'Runtime': u'164',
     u'Title': u'Dark Knight Rises, The (2012)'},
    u'_type': u'movies',
    u'highlight': {u'Title': [u'<em>Dark</em> <em>Knight</em> <em>Rises</em>, The (2012)']}}],
  u'max_score': 8.52379,
  u'tot

In [23]:
for hit in response['hits']['hits']:
    print hit['highlight']

{u'Title': [u'<em>Dark</em> <em>Knight</em> <em>Rises</em>, The (2012)']}


In [15]:
print response

{u'hits': {u'hits': [{u'_type': u'movies', u'_source': {u'Rating': 8.5, u'Genres': u'Action, Thriller', u'Language': u'English, Arabic', u'Title': u'Dark Knight Rises, The (2012)', u'Country': u'UK, USA', u'Director': u'Christopher Nolan', u'Cast': u'Christian Bale (Bruce Wayne), Gary Oldman (Commissioner Gordon), Tom Hardy (Bane), Joseph Gordon-Levitt (Blake), Anne Hathaway (Selina)', u'Plot': u"Despite his tarnished reputation after the events of The Dark Knight, in which he took the rap for Dent's crimes, Batman feels compelled to intervene to assist the city and its police force which is struggling to cope with Bane's plans to destroy the city.", u'Runtime': u'164'}, u'_score': 8.52379, u'_index': u'emdb', u'highlight': {u'Title': [u'<em>Dark</em> <em>Knight</em> <em>Rises</em>, The (2012)']}, u'_id': u'592572248fbb650bf6cea515'}], u'total': 1, u'max_score': 8.52379}, u'_shards': {u'successful': 5, u'failed': 0, u'total': 5}, u'took': 4, u'timed_out': False}


In [15]:
es.search(index="emdb", body={"query": {"prefix" : { "Director" : "Chr" }}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [], u'max_score': None, u'total': 0},
 u'timed_out': False,
 u'took': 3}

In [17]:
es.search(index="emdb", body={"query": 
{"fuzzy_like_this_field" : { "Title" : 
{"like_text": "the godfahter", "max_query_terms":5}}}})

RequestError: TransportError(400, u'parsing_exception', u'no [query] registered for [fuzzy_like_this_field]')